In [1]:
# coding=utf-8
# Copyright 2023-present the HuggingFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image

from tqdm import tqdm
import csv

In [2]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
dataset = load_dataset("csv", data_files={"train" : "./ambiguous_questions_train.csv", "test" : "./ambiguous_questions_test.csv"})

Using custom data configuration default-b4c98d92954de12c
Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-b4c98d92954de12c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from open_flamingo import create_model_and_transforms

model, image_processor, tokenizer = create_model_and_transforms(
    clip_vision_encoder_path="ViT-L-14",
    clip_vision_encoder_pretrained="openai",
    lang_encoder_path="anas-awadalla/mpt-7b",
    tokenizer_path="anas-awadalla/mpt-7b",
    cross_attn_every_n_layers=4
)

Using pad_token, but it is not set yet.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Flamingo model initialized with 1384781840 trainable parameters


In [5]:
# grab model checkpoint from huggingface hub
from huggingface_hub import hf_hub_download
import torch

checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-9B-vitl-mpt7b", "checkpoint.pt")
model.load_state_dict(torch.load(checkpoint_path), strict=False)

_IncompatibleKeys(missing_keys=['vision_encoder.class_embedding', 'vision_encoder.positional_embedding', 'vision_encoder.proj', 'vision_encoder.conv1.weight', 'vision_encoder.ln_pre.weight', 'vision_encoder.ln_pre.bias', 'vision_encoder.transformer.resblocks.0.ln_1.weight', 'vision_encoder.transformer.resblocks.0.ln_1.bias', 'vision_encoder.transformer.resblocks.0.attn.in_proj_weight', 'vision_encoder.transformer.resblocks.0.attn.in_proj_bias', 'vision_encoder.transformer.resblocks.0.attn.out_proj.weight', 'vision_encoder.transformer.resblocks.0.attn.out_proj.bias', 'vision_encoder.transformer.resblocks.0.ln_2.weight', 'vision_encoder.transformer.resblocks.0.ln_2.bias', 'vision_encoder.transformer.resblocks.0.mlp.c_fc.weight', 'vision_encoder.transformer.resblocks.0.mlp.c_fc.bias', 'vision_encoder.transformer.resblocks.0.mlp.c_proj.weight', 'vision_encoder.transformer.resblocks.0.mlp.c_proj.bias', 'vision_encoder.transformer.resblocks.1.ln_1.weight', 'vision_encoder.transformer.resbloc

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)
model.eval()


Flamingo(
  (vision_encoder): VisionTransformer(
    (patchnorm_pre_ln): Identity()
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): Layer

In [7]:
from collections import Counter

with open("./ambiguous_questions_train.csv", 'r') as f:
    reader = csv.reader(f)
    train_lines = [line for line in reader]


with open("./ambiguous_questions_test.csv", 'r') as f:
    reader = csv.reader(f)
    test_lines = [line for line in reader]
    
    
  
num_few_shot_examples = 32
#promt = f"Instructions : Classify the following into ambigous and definite question. An ambiguous question is unanswerable question considering image. "

few_shot_image_ids = [train_lines[idx][5] for idx in range(1,num_few_shot_examples + 1)]
few_shot_images = [Image.open("./images/" + str(image_id) + ".jpg").convert('RGB') for image_id in few_shot_image_ids]

vision_x = [image_processor(demo_image).unsqueeze(0) for demo_image in few_shot_images]

labels = ["ambiguous" if train_lines[idx][6] == "O" else "definite" for idx in range(1, 101)]

few_shot_examples_ids = []
counter_list = []
a_count = 0
d_count = 0
for idx, label in enumerate(labels):
    if a_count + d_count == num_few_shot_examples:
        break
    
    if label == "ambiguous":
        if a_count < 16:
            few_shot_examples_ids.append(idx)
            a_count += 1
            counter_list.append(label)
    else:
        
        if d_count < 16:
            few_shot_examples_ids.append(idx)
            d_count += 1
            
            counter_list.append(label)
            
assert num_few_shot_examples == len(few_shot_examples_ids)

print(Counter(counter_list))
    

tokenizer.padding_side = "left" # For generation padding tokens should be on the left

text = [f"The question of <image> is '{train_lines[idx+1][1]}'. Is the question is ambiguous or definite? Answer: {labels[idx]} <|endofchunk|>"  for idx in few_shot_examples_ids]
text = "".join(text)

print(text)


Counter({'definite': 16, 'ambiguous': 16})
The question of <image> is 'Is the pillowcase to the right of the white thing that is to the right of the lamp?'. Is the question is ambiguous or definite? Answer: definite <|endofchunk|>The question of <image> is 'Is the traffic cone to the left of the bicycle that is lying in the gravel?'. Is the question is ambiguous or definite? Answer: definite <|endofchunk|>The question of <image> is 'Which color is the train the railroad contains?'. Is the question is ambiguous or definite? Answer: ambiguous <|endofchunk|>The question of <image> is 'Is the kneeling man holding a phone?'. Is the question is ambiguous or definite? Answer: definite <|endofchunk|>The question of <image> is 'Are the chairs in the bottom or in the top part of the photo?'. Is the question is ambiguous or definite? Answer: ambiguous <|endofchunk|>The question of <image> is 'Is the vegetable to the left of the sausage smooth and red?'. Is the question is ambiguous or definite? A

In [8]:
predictions = []
for idx, line in tqdm(enumerate(test_lines)):
    if idx == 0:
        continue
    question = line[1]
    image_id = line[5]
    image = Image.open("./images/" + str(image_id) + '.jpg').convert('RGB')
    input_prompt = text + f"The question of <image> is '{question}'. Is the question is ambiguous or definite? Answer:"
    
    # print(input_prompt)
    
    input_images = vision_x + [image]
    input_images = torch.cat(vision_x, dim=0)
    input_images = input_images.unsqueeze(1).unsqueeze(0)
    
    lang_x = tokenizer(
        [input_prompt] ,
        return_tensors="pt",
    )
    
    generated_text = model.generate(
    vision_x=input_images.to(device),
    lang_x=lang_x["input_ids"].to(device),
    attention_mask=lang_x["attention_mask"].to(device),
    max_new_tokens=2,
    num_beams=4,
    )
    
    #print(tokenizer.batch_decode(lang_x["input_ids"],skip_special_tokens=False))
    #print("Input prompt: ", input_prompt)
    
    decoded_text =  tokenizer.decode(generated_text[0], skip_special_tokens=True)
    if idx < 10:
        print("question: ", question)
        print("Generated text: ",decoded_text)
        
    answer = decoded_text.split("Answer:")[-1].strip()

    #inputs = processor(image, input_promt, return_tensors="pt").to(device)
    
    #out = model.generate(**inputs)
    #prediction = processor.decode(out[0], skip_special_tokens=True)
    #print(prediction)
    predictions.append(answer)
    
with open ("./test_fewshot_flamingo_balance.csv", 'w') as f:
        
    writer = csv.writer(f)
    for idx, line in enumerate(test_lines):
        if idx == 0:
            writer.writerow(line)
        else:
            line.append(predictions[idx-1])
            writer.writerow(line)
                
            



0it [00:00, ?it/s]/opt/conda/envs/torch2.0/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
2it [00:11,  5.71s/it]

question:  Is the meat on top of the egg grilled or raw?
Generated text:  The question of  is 'Is the pillowcase to the right of the white thing that is to the right of the lamp?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Is the traffic cone to the left of the bicycle that is lying in the gravel?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Which color is the train the railroad contains?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the kneeling man holding a phone?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Are the chairs in the bottom or in the top part of the photo?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the vegetable to the left of the sausage smooth and red?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the grass that is not short tall or shor

Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
3it [00:21,  7.57s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


question:  What kind of food is on the plate that is to the left of the tray?
Generated text:  The question of  is 'Is the pillowcase to the right of the white thing that is to the right of the lamp?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Is the traffic cone to the left of the bicycle that is lying in the gravel?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Which color is the train the railroad contains?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the kneeling man holding a phone?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Are the chairs in the bottom or in the top part of the photo?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the vegetable to the left of the sausage smooth and red?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the grass that is n

4it [00:31,  8.52s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


...

9it [01:23, 10.10s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


question:  How big is the boat on the left?
Generated text:  The question of  is 'Is the pillowcase to the right of the white thing that is to the right of the lamp?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Is the traffic cone to the left of the bicycle that is lying in the gravel?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Which color is the train the railroad contains?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the kneeling man holding a phone?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Are the chairs in the bottom or in the top part of the photo?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the vegetable to the left of the sausage smooth and red?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the grass that is not short tall or short?'. Is the q

10it [01:33, 10.21s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


question:  Does the man near the bag buy the apples in the middle?
Generated text:  The question of  is 'Is the pillowcase to the right of the white thing that is to the right of the lamp?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Is the traffic cone to the left of the bicycle that is lying in the gravel?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Which color is the train the railroad contains?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the kneeling man holding a phone?'. Is the question is ambiguous or definite? Answer: definite The question of  is 'Are the chairs in the bottom or in the top part of the photo?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the vegetable to the left of the sausage smooth and red?'. Is the question is ambiguous or definite? Answer: ambiguous The question of  is 'Is the grass that is not short ta

11it [01:44, 10.28s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
12it [01:54, 10.33s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
13it [02:05, 10.37s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
14it [02:15, 10.38s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
15it [02:25, 10.39s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
16it [02:36, 10.40s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
17it [02:46, 10.41s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
18it [02:57, 10.40s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
19it [03:07, 10.40s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
20it [03:18, 10.41s/it]Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
21it [03:28, 10.42s/it]Setting `pad_token_id` to `